# 决策树：
是一种基本的分类与回归方法，此处主要讨论分类的决策树

在分类问题中，表示基于特征对实例进行分类的过程，可以认为是if-then的集合，也可以认为是定义在特征空间与类空间上的条件概率分布。

# 决策树通常有三个步骤：
特征选择、决策树的生成、决策树的修剪。

用决策树分类：从根节点开始，对实例的某一特征进行测试，根据测试结果将实例分配到其子节点，此时每个子节点对应着该特征的一个取值，如此递归的对实例进行测试并分配，直到到达叶节点，最后将实例分到叶节点的类中。

![title](pic/8.png)

# 决策树的构造
决策树学习的算法通常是一个递归地选择最优特征，并根据该特征对训练数据进行分割，使得各个子数据集有一个最好的分类的过程。这一过程对应着对特征空间的划分，也对应着决策树的构建。

1） 开始：构建根节点，将所有训练数据都放在根节点，选择一个最优特征，按着这一特征将训练数据集分割成子集，使得各个子集有一个在当前条件下最好的分类。

2） 如果这些子集已经能够被基本正确分类，那么构建叶节点，并将这些子集分到所对应的叶节点去。

3）如果还有子集不能够被正确的分类，那么就对这些子集选择新的最优特征，继续对其进行分割，构建相应的节点，如果递归进行，直至所有训练数据子集被基本正确的分类，或者没有合适的特征为止。

4）每个子集都被分到叶节点上，即都有了明确的类，这样就生成了一颗决策树。
# 决策树的特点：

优点：计算复杂度不高，输出结果易于理解，对中间值的缺失不敏感，可以处理不相关特征数据

缺点：可能会产生过度匹配的问题

适用数据类型：数值型和标称型


伪代码：

If so return 类标签：

Else

     寻找划分数据集的最好特征
    
     划分数据集
        
     创建分支节点
    
         for 每个划分的子集
        
             调用函数createBranch()并增加返回结果到分支节点中
            
         return 分支节点


# 熵：不确定性
为了计算熵，我们需要计算所有类别所有可能值所包含的信息期望值，通过下式得到：


$$
H=-\Sigma_{i=1}^{n} p\left(x_{i}\right) \log _{2} p\left(x_{i}\right)
\\
$$

训练数据集D，则训练数据集D的经验熵为H(D)，|D|表示其样本容量，及样本个数。设有K个类Ck，k = 1,2,3,···,K，|Ck|为属于类Ck的样本个数，这经验熵公式可以写为：
$$
H(D)=-\Sigma \frac{\left|c_{k}\right|}{|D|} \log _{2} \frac{\left|c_{k}\right|}{|D|}
\\
$$

比如，在15个人中，有9个贷款，6个不贷款 ，那么熵H（D）为：
$$
H(D)=-\frac{9}{15} \log _{2} \frac{9}{15}-\frac{6}{15} \log _{2} \frac{6}{15}=0.971
$$
![title](pic/9.png)

# 信息增益 ：
划分数据集的大原则是：将无序数据变得更加有序，但是各种方法都有各自的优缺点，信息论是量化处理信息的分支科学，在划分数据集前后信息发生的变化称为信息增益，获得信息增益最高的特征就是最好的选择，所以必须先学习如何计算信息增益，集合信息的度量方式称为香农熵，或者简称熵。
# 条件熵：

信息增益表示得知特征X的信息而使得类Y的信息不确定性减少的程度。

条件熵H(Y∣X) H(Y|X)H(Y∣X)表示在已知随机变量X的条件下随机变量Y的不确定性，随机变量X给定的条件下随机变量Y的条件熵(conditional entropy) H(Y|X)，定义X给定条件下Y的条件概率分布的熵对X的数学期望：
$$
H(Y | X)=\sum_{i=1}^{n} p_{i} H\left(Y | X=x_{i}\right)\\
p_{i}=P\left(X=x_{i}\right)
\\
$$

信息增益是相对于特征而言的。所以，特征A对训练数据集D的信息增益g(D,A)，定义为集合D的经验熵H(D)与特征A给定条件下D的经验条件熵H(D|A)之差，即：
$$
g(D, A)=H(D)-H(D | A)
\\
$$

In [1]:
import numpy as np

In [2]:
def createDataSet():   #简单数据集做测试
    dataSet = [[1, 1, 'yes'], [1, 1, 'yes'], [1, 0, 'no'], [0, 1, 'no'], [0, 1, 'no']]
    labels = ['no surfacing', 'flippers']
    return dataSet,labels

In [15]:
def creatDataSet2():
    # 数据集
    dataSet=[[0, 0, 0, 0, 'no'],
            [0, 0, 0, 1, 'no'],
            [0, 1, 0, 1, 'yes'],
            [0, 1, 1, 0, 'yes'],
            [0, 0, 0, 0, 'no'],
            [1, 0, 0, 0, 'no'],
            [1, 0, 0, 1, 'no'],
            [1, 1, 1, 1, 'yes'],
            [1, 0, 1, 2, 'yes'],
            [1, 0, 1, 2, 'yes'],
            [2, 0, 1, 2, 'yes'],
            [2, 0, 1, 1, 'yes'],
            [2, 1, 0, 1, 'yes'],
            [2, 1, 0, 2, 'yes'],
            [2, 0, 0, 0, 'no']]
    #分类属性
    labels=['年龄','有工作','有自己的房子','信贷情况']
    #返回数据集和分类属性
    return dataSet,labels

In [16]:
def systemShannonEnt(dataset):   #计算传入数据的熵
    numEntries = len(dataset)
    labelCounts = {}
    for featVec in dataset:
        currentlabel = featVec[-1]
        if currentlabel not in labelCounts.keys():
            labelCounts[currentlabel] = 0
        labelCounts[currentlabel] += 1
#         labelCounts[currentlabel] =labelCounts.get(currentlabel,0)+1
    ShannonEnt = 0.0
    for key in labelCounts.keys():  #迭代key
        prob = float(labelCounts[key] / numEntries)
        ShannonEnt -= prob * np.log2(prob)    #np.log的参数已经改变，用log2（x），或者np.math.log(x,2)
    return ShannonEnt

In [18]:
a,b=creatDataSet2()
systemShannonEnt(a)

0.9709505944546686

In [6]:
def splitDataSet(dataset,axis,value):  #传入的数据，划分依据的特征，特征的值
    retDataSet=[]
    for featVec in dataset:
        if featVec[axis]==value:
            reducedFeatVec=featVec[:axis]
            reducedFeatVec.extend(featVec[axis+1:])
            retDataSet.append(reducedFeatVec)
    return  retDataSet   #返回的数据是不含axis的

In [7]:
splitDataSet(a,0,1)

[[1, 'yes'], [1, 'yes'], [0, 'no']]

In [8]:
def chooseBestFeatureToSplit(dataSet):  #选出最佳分割特征
    numFeatures=len(dataSet[0])-1
    baseEntropy=systemShannonEnt(dataSet) #计算基础熵
    bestInfoGain=0.0 ;bestFeature=-1
    for i in range(numFeatures):
        featList=[ex[i] for ex in  dataSet]   #第i列特征
        uniqueVals=set(featList)              #特征取值集合
        newEnt=0.0
        for value in uniqueVals:             #计算熵
            subDataSet=splitDataSet(dataSet,i,value)
            prob=len(subDataSet)/float(len(dataSet))
            newEnt+=prob*systemShannonEnt(subDataSet)
        InfoGain=baseEntropy-newEnt       #熵增
        if InfoGain>bestInfoGain:        #挑选熵增最大的特征
            bestInfoGain=InfoGain
            bestFeature=i
    return bestFeature

In [9]:
chooseBestFeatureToSplit(splitDataSet(a,1,0))

-1

In [10]:
def majorityCnt(classList):
    classCount={}
    for vote in classList:
        if vote not in classCount.keys() :
            classCount[vote]=0
        classCount[vote]+=1
#         classCount[vote] = classCount.get(vote,0)+1
    sortedClassCount=sorted(classCount.items(),key=operator.itemgetter(1),reverse=True)
    return sortedClassCount[0][0]


In [11]:
def createTree(dataSet,labels):  #构建决策树
    classList=[ex[-1] for ex in dataSet]   #label
    if classList.count(classList[0])==len(classList):  #如果只剩一个label，类别完全相同，停止划分
        return classList[0]
    if len(dataSet[0])==1:           #遍历完所有的特征，返回出现次数最多的
        return majorityCnt(classList)
    bestFeat=chooseBestFeatureToSplit(dataSet)
    bestFeatLabel=labels[bestFeat]  #mappnig
    myTree={bestFeatLabel:{}}
    del (labels[bestFeat])
    featValues=[ex[bestFeat] for ex in dataSet]
    uniquevals=set(featValues)
    for value in uniquevals:
        myTree[bestFeatLabel][value]=createTree(splitDataSet(dataSet,bestFeat,value),labels)
    return myTree

In [19]:
createTree(a,b)

{'有自己的房子': {0: {'有工作': {0: 'no', 1: 'yes'}}, 1: 'yes'}}